# pYPK0_RPL17A_EcacpH_RPL16B

This notebook describes the assembly of the [_Saccaromyces cerevisiae_](www.yeastgenome.org)
single gene expression plasmid vector `pYPK0_RPL17A_EcacpH_RPL16B`.

It is made by _in-vivo_ [homologous recombination](https://en.wikipedia.org/wiki/Homologous_recombination) between three PCR products and one linear vector fragment. The PCR products are:

- a promoter generated from a pYPK_Z vector
- a gene from a pYPKa_A vector
- a terminator from a pYPKa_E vector

The three PCR products are joined with a [pYPKpw](https://github.com/BjornFJohansson/ypk-xylose-pathways/blob/master/notebooks/pYPKpw.ipynb) 
plasmid that has a [URA3](http://www.yeastgenome.org/locus/S000000747/overview) marker for selection in yeast and a _S. crevisiae_ [2 micron](http://blog.addgene.org/plasmids-101-yeast-vectors) origin of replication.

![pYPK0_promoter_gene_terminator](TUplasmid.png "pYPK0_promoter_gene_terminator")


In [34]:
from pydna.parsers import parse_primers
from pydna.readers import read
from pydna.amplify import pcr
from pydna.assembly import Assembly

The Yeast Pathway Kit [standard primers](standard_primers.txt) are read into a dictionary in the code cell below.

In [35]:
p = { x.id: x for x in parse_primers("standard_primers.txt") }

The backbone vector [pYPKpw](pYPKpw.gb) is read from a local file in the code cell below.

In [36]:
pYPKpw = read("pYPKpw.gb")

The restriction enzyme functionality is provided by [biopython](http://biopython.org).

In [37]:
from Bio.Restriction import EcoRV

The backbone vector is linearized by digestion with restriction enzyme [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html).

In [38]:
pYPK0_EcoRV = pYPKpw.linearize(EcoRV)

The pYPKa derived _E. coli_ plasmids containing [promoter](pYPKa_Z_RPL17A.gb), [gene](pYPKa_A_EcacpH.gb) and [terminator](pYPKa_E_RPL16B.gb)
are read into three variables below.

In [39]:
promoter_template   = read("pYPKa_Z_RPL17A.gb")
gene_template       = read("pYPKa_A_EcacpH.gb")
terminator_template = read("pYPKa_E_RPL16B.gb")

The construction of the three vector above are described in the [pYPKa_ZE_RPL17A](pYPKa_ZE_RPL17A.ipynb) and [pYPKa_A_EcacpH](pYPKa_A_EcacpH.ipynb) notebooks.

Three DNA fragments are PCR amplified using [standard primers](standard_primers.txt).

In [40]:
prom = pcr( p['577'], p['567'], promoter_template)
gene = pcr( p['468'], p['467'], gene_template)
term = pcr( p['568'], p['578'], terminator_template)

In [41]:
prom.name = "RPL17A_promoter"
gene.name = "EcacpH"
term.name = "RPL16B_terminator"

The four linear DNA fragments are mixed and transformed to a _Saccharomyces cerevisiae_ ura3 mutant.

The fragments will be assembled by _in-vivo_ [homologous recombination](http://www.ncbi.nlm.nih.gov/pubmed/2828185) since they carry short stretches of homologous DNA . 

The `pydna.assemble.Assembly` class simulates homologous recombination between DNA sequences.

In [42]:
asm = Assembly( (pYPK0_EcoRV, prom, gene, term), limit=31 )

asm

Assembly
fragments..: 5603bp 900bp 671bp 1038bp
limit(bp)..: 31
G.nodes....: 8
algorithm..: common_sub_strings

The `limit` property sets the lower length limit for considering DNA homology between fragments. 

In [43]:
candidates = asm.assemble_circular()

Two circular assembly candidates of equel size are expected, representing the reverse complement of each other. **More than one circular products might indicate an incorrect assembly strategy or represent by-products that arise in the assembly process.** 

We get two circular products for the pYPK0_RPL17A_EcacpH_RPL16B vector.

In [44]:
candidates

[Contig(o7740), Contig(o7740)]

In [45]:
candidate = candidates[0]

We can visualize the recombination like so:

In [46]:
candidate.figure()

 -|pYPKpw_lin|124
|             \/
|             /\
|             124|RPL17A_promoter|50
|                                 \/
|                                 /\
|                                 50|EcacpH|37
|                                           \/
|                                           /\
|                                           37|RPL16B_terminator|242
|                                                                \/
|                                                                /\
|                                                                242-
|                                                                   |
 -------------------------------------------------------------------

We synchronize the vector to the 577 PCR primer. This means that the plasmid origin is shifted so that it starts with the 577 primer. This is for convention only.

In [47]:
result = candidate.synced(p["577"])

The circular `cseguid` checksum identifies 

In [48]:
result.cseguid()

Cx4gG62Az4KkHgUHUsj3XGZVjqc

In [49]:
result.description = "pYPK0_RPL17A_EcacpH_RPL16B"

In [50]:
result.stamp()

cSEGUID_Cx4gG62Az4KkHgUHUsj3XGZVjqc

In [51]:
result.write("pYPK0_RPL17A_EcacpH_RPL16B.gb")

pYPK0_RPL17A_EcacpH_RPL16B.gb